In [ ]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from src.datasets import VoxCelebDataModule
from src.models import ResNet34

seed_everything(42, workers=True)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min"
)

In [ ]:
dm = VoxCelebDataModule(batch_size=4)
model = ResNet34(num_classes=5)
logger = TensorBoardLogger("tb_logs", name="resnet34")
trainer = Trainer(
    deterministic=True,
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm)